## This notebook inputs the missing values using KNN

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [2]:
INPUT, OUTPUT = 'output/CompleteWeeklyIndexes', 'output/KNNWeeklyImputed'

In [3]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data

In [4]:
df_test.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2020-02-21,3366.854980,3.813365e+09,15.462500,0.0,29198.152344,2.740575e+08,9605.762695,2.553652e+09,23396.726172,60100.000000,...,27612.716016,1.566489e+09,1173.309985,208087340.0,NaN,4.341429,NaN,NaN,NaN,NaN
2020-02-28,3080.693946,6.307054e+09,31.942000,0.0,26635.150000,6.038540e+08,8737.351953,3.147614e+09,22030.697754,114950.000000,...,26663.830078,1.854099e+09,1080.093994,449170340.0,24.443546,29.937143,76.532662,NaN,NaN,NaN
2020-03-06,3044.006054,5.979102e+09,36.757999,0.0,26339.529688,5.638040e+08,8724.750195,4.040054e+09,21121.148047,104200.000000,...,26342.621875,2.292851e+09,1030.441992,434007680.0,NaN,26.794286,NaN,NaN,NaN,NaN
2020-03-13,2712.365967,8.104234e+09,57.792001,0.0,23361.727734,7.641180e+08,7917.065820,4.597718e+09,18994.524219,172440.000000,...,24801.312110,3.955053e+09,860.319995,722226640.0,NaN,57.690000,NaN,NaN,NaN,NaN
2020-03-20,2437.806641,8.298607e+09,78.350001,0.0,20441.606771,8.115167e+08,7223.170085,4.795060e+09,16913.373047,178033.333333,...,23163.650391,3.852502e+09,753.783346,627243300.0,NaN,52.960000,NaN,NaN,NaN,NaN


In [5]:
#Scaled data has zero mean and unit variance
scaler = preprocessing.StandardScaler()
scaler.fit(df)
array = scaler.transform(df)
array_test = scaler.transform(df_test)
df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
df_scaled.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,0.516845,-1.173684,0.458946,0.0,-0.097348,-0.398262,1.741088,-0.867106,2.026643,-1.613284,...,-0.301548,-1.182157,1.292603,-1.108292,NaN,-0.478968,NaN,NaN,NaN,NaN
2000-01-14,0.622367,-1.195971,0.084189,0.0,0.052938,-0.219366,1.822917,-0.834847,2.142257,-1.613284,...,-0.393082,-1.182157,1.340506,-1.108292,NaN,-0.540299,NaN,NaN,NaN,NaN
2000-01-21,0.625514,-1.144887,0.057589,0.0,-0.024923,-0.201874,2.060511,-0.411409,2.225496,-1.613284,...,-0.459789,-1.182157,1.345545,-1.108292,NaN,-0.706121,NaN,NaN,NaN,NaN
2000-01-28,0.421959,-1.149579,0.340176,0.0,-0.223574,-0.227466,1.828617,-0.353183,2.244522,-1.613284,...,-0.425608,-1.182157,1.284557,-1.108292,0.365881,-0.101894,NaN,0.508212,0.367186,1.245141
2000-02-04,0.487425,-1.191452,0.233777,0.0,-0.212513,-0.472543,1.981936,-0.916105,2.400759,-1.613284,...,-0.389843,-1.182157,1.418557,-1.108292,NaN,0.297896,-0.796131,NaN,NaN,NaN


In [6]:
imp = KNNImputer(n_neighbors=5, weights="uniform")
imp.fit(df_scaled)
df_imputed = imp.transform(df_scaled)
print(df_imputed.shape)
df_test_imputed = imp.transform(df_test_scaled)
df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)

(782, 22)


In [7]:
df_final.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2014-11-28,2.935109,0.025779,-0.941165,0.0,2.831675,-1.732824,2.654640,-1.413322,1.665521,0.601343,...,1.286594,1.391380,0.636944,1.799074,-0.853205,0.138889,0.830879,-0.804975,-0.431014,-0.640274
2014-12-05,2.931809,0.483129,-0.919964,0.0,2.859917,-1.677655,2.653393,-0.297913,1.790516,0.447798,...,1.223253,2.589543,0.660697,1.881810,-0.963481,-0.478968,-0.057824,-0.879996,-0.150168,-1.019271
2014-12-12,2.814278,0.697536,-0.347030,0.0,2.738337,-1.434433,2.584880,0.038177,1.725571,0.840400,...,1.196135,1.800954,0.520673,1.637239,-1.012729,-0.515312,0.904611,0.007147,-0.041532,-0.925040
2014-12-19,2.757041,1.397868,-0.178772,0.0,2.660383,-0.649793,2.517461,1.559074,1.575819,0.899426,...,1.058196,1.148853,0.404031,3.665161,-0.840741,0.011683,0.456847,0.981432,0.172146,-0.843945
2014-12-26,2.985915,-0.327093,-0.697944,0.0,2.918445,-1.879237,2.637255,-1.912536,1.798050,0.005403,...,1.163534,0.760872,0.604114,0.438438,-0.807252,1.749404,0.449973,-0.934712,0.344659,-0.340244


In [8]:
# Descaling the data
array_2 = scaler.inverse_transform(df_final)
df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

array_test_2 = scaler.inverse_transform(df_test_final)
df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)

df_final2.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,1420.333984,1.068760e+09,25.016000,0.0,11250.781836,182562000.0,3542.894043,1.598166e+09,18476.772461,0.0,...,16169.606250,0.0,949.868006,0.0,29.716552,0.252857,35.295035,109.690163,71.576902,0.030615
2000-01-14,1448.648023,1.033940e+09,21.684000,0.0,11587.958008,196256000.0,3611.343994,1.609134e+09,18829.544922,0.0,...,15720.128125,0.0,957.357996,0.0,29.716552,0.214286,39.567372,109.690163,71.576902,0.030615
2000-01-21,1449.492493,1.113750e+09,21.447500,0.0,11413.272461,197595000.0,3810.092468,1.753105e+09,19083.530078,0.0,...,15392.563867,0.0,958.146008,0.0,26.837567,0.110000,43.306741,70.758729,68.484446,0.030909
2000-01-28,1394.874023,1.106420e+09,23.960000,0.0,10967.587890,195636000.0,3616.111963,1.772902e+09,19141.585938,0.0,...,15560.411914,0.0,948.610010,0.0,24.412899,0.490000,35.092878,71.914247,117.577146,0.033573
2000-02-04,1412.439990,1.041000e+09,23.014001,0.0,10992.404102,176876000.0,3744.364014,1.581506e+09,19618.308203,0.0,...,15736.035157,0.0,969.562000,0.0,26.381458,0.741429,34.391162,63.131003,66.874997,0.030877


In [10]:
df_final2.to_csv(OUTPUT+'_training.csv', index = True)
df_test_final2.to_csv(OUTPUT+ '_test.csv', index = True)